* 본 커널은 아래 커널들을 참고했습니다. 
    - https://www.kaggle.com/robikscube/ieee-fraud-detection-first-look-and-eda

## 데이터셋 임포트하기

In [ ]:
# https://github.com/mouradmourafiq/pandas-summary
!pip install pandas-summary

In [ ]:
from pandas_summary import DataFrameSummary

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import os, timeit, datetime

pd.set_option('display.max_columns', 400)

# for local execution
current_dir = os.path.abspath('.')
dataset_dir = os.path.join(os.path.abspath(current_dir + "/../"), 'dataset_kaggle_IEEE_CIS_Fraud_Detection')
print(os.listdir(dataset_dir))

In [ ]:
%%time
# 데이터 로딩(for Kaggle Kernel)
# train_tran = pd.read_csv('../input/train_transaction.csv', index_col='TransactionID')
# train_iden = pd.read_csv('../input/train_identity.csv', index_col='TransactionID')
# test_tran = pd.read_csv('../input/test_transaction.csv', index_col='TransactionID')
# test_iden = pd.read_csv('../input/test_identity.csv', index_col='TransactionID')
# sample_sub = pd.read_csv('../input/sample_submission.csv', index_col='TransactionID')

# 데이터 로딩 (for local execution)
train_tran = pd.read_csv(os.path.join(dataset_dir, 'train_transaction.csv'))
train_iden = pd.read_csv(os.path.join(dataset_dir, 'train_identity.csv'))
test_tran = pd.read_csv(os.path.join(dataset_dir, 'test_transaction.csv'))
test_iden = pd.read_csv(os.path.join(dataset_dir, 'test_identity.csv'))
sample_sub = pd.read_csv(os.path.join(dataset_dir, 'sample_submission.csv'))

In [ ]:
%%time
dfs = DataFrameSummary(train_tran)

In [ ]:
%%time
dfs.columns_types

In [ ]:
%%time
dfs.columns_stats

Train과 Test 데이터는 모두 각각 Identity와 Transaction 두 개의 .csv파일로 나뉘어 있습니다. 이는 'TransactionID'라는 column으로 join되어 있습니다. 단, `train_identity.csv`와 `train_transaction.csv`의 모든 거래 내역(transactions)들이 서로 완벽히 일치하지는 않습니다. 

* <b>`train_transaction.csv`의 범주형 번수</b>
    * TransactionID : reference datetime(실제 timestamp와 다름)으로부터의 timedelta(시간변화량)을 의미
    * TransactionAmt : USD를 이용해 지불된 거래 금액
    * ProductCD : 제품 code(각 거래마다의 제품 코드)
    * card1 ~ card6 : 거래에 사용한 카드 정보(카드 타입, 카드 카테고리, 카드 발행은행, 국가 등)
    * addr1 ~ addr2 : 주소
    * dist : 거리(distance)
    * P_emaildomain, R_emaildomain : 구매자 및 수신자 이메일 도메인 주소
    * C1 ~ C14 : 얼마나 많은 주소가 지불 카드와 관련있는지에 대한 횟수(counting)로, 실제 의미는 비식별 처리되어(masked)있음.
    * D1 ~ D15 : 이전 거래일과의 시간변화량(timedelta)
    * M1 ~ M9 : 카드에 적힌 이름이나 주소 등등이 일치하는지 여부


* <b>`train_identity.csv`의 범주형 변수</b>
    * DeviceType
    * DeviceInfo
    * id_12 ~ id_38 : 트랜잭션과 관련된 식별 정보 (네트워크 연결 정보 - IP, ISP, Proxy 등) 및 디지털 서명(UA/browser/OS/version 등)을 의미하며, Vesta의 사기 방지 시스템 및 디지털 보안 파트너사가 함께 수집했습니다. 필드의 이름들은 개인정보 보호를 위해 비식별 처리되어 있습니다. 
   
이번 대회의 데이터는 메모리 문제를 유발할 수 있기 때문에, Memory reduce를 진행하여야 합니다. 불러오는 것은 물론 처리에도 시간이 많이 소비되므로, 메모리 사용량을 줄여 보겠습니다. 이번 대회의 데이터 타입들은 대부분 int64, float64로 이루어져 있으며, 실제 데이터는 int64 타입이지만 실질적인 데이터 범위가 int16에만 속한다면 메모리를 줄이는 것이 효율적일 것입니다. (https://hwiyong.tistory.com/238 참고)


#### 데이터셋의 메모리 줄이기 

In [ ]:
%%time
## Dataframe의 메모리 사이즈를 줄이기 위한 함수
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## REducing memory
train_tran = reduce_mem_usage(train_tran)
train_iden = reduce_mem_usage(train_iden)
test_tran = reduce_mem_usage(test_tran)
test_iden = reduce_mem_usage(test_iden)

In [ ]:
train_tran.head()

In [ ]:
train_tran.info()

In [ ]:
train_iden.head()

In [ ]:
train_iden.info()

## Explore Data 
첫번째로 Train과 Test 데이터에 대한 time series 분할을 해볼 것입니다. `TransactionDT` feature는 주어진 참조 시간(reference datetime, 현실의 시간과 다름)에 대한 시간 변화량(timedelta)입니다. train과 test 데이터가 시간에 따라 분리되어 있으며, 두 데이터셋 사이에 약간 간격이 존재하는 것을 확인할 수 있습니다. 이는 추후 CV(Cross validation)기법을 사용해야 하는지에 대한 여부 결정에 영향을 미칩니다. 이제 다른 train, test의 feature들에 대해서도 분석해 보겠습니다.

### train_transaction.csv의 정수형 데이터 피쳐들 살펴보기
#### Training set에 들어 있는 Target의 분포 살펴보기

In [ ]:
train_tran.groupby('isFraud') \
    .count()['TransactionID'] \
    .plot(kind='barh',
          title='Distribution of Target in Train',
          figsize=(15, 3))
plt.show()

print('train_tran.df안에 존재하는 사기 거래의 비율 : ', train_tran['isFraud'].mean() * 100)

#### 거래금액(`TransactionAmt`)의 분산 확인해보기
매우 큰 트랜잭션이 전체를 흔드는 것을 방지하기 위해 log 정규화를 수행한 후 plot을 그려 봅니다. 로그로 변환하는 과정 때문에 0~1 사이의 모든 값은 음수로 표시됩니다.
이제 정상 거래와 사기 거래에 대해서도 plot을 그려 보겠습니다. 

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 6))
train_tran.loc[train_tran['isFraud'] == 1] \
    ['TransactionAmt'].apply(np.log) \
    .plot(kind='hist',
          bins=100,
          title='Log Transaction Amt - Fraud',
          xlim=(-3, 10),
         ax= ax1)
train_tran.loc[train_tran['isFraud'] == 0] \
    ['TransactionAmt'].apply(np.log) \
    .plot(kind='hist',
          bins=100,
          title='Log Transaction Amt - Not Fraud',
          xlim=(-3, 10),
         ax=ax2)
train_tran.loc[train_tran['isFraud'] == 1] \
    ['TransactionAmt'] \
    .plot(kind='hist',
          bins=100,
          title='Transaction Amt - Fraud',
         ax= ax3)
train_tran.loc[train_tran['isFraud'] == 0] \
    ['TransactionAmt'] \
    .plot(kind='hist',
          bins=100,
          title='Transaction Amt - Not Fraud',
         ax=ax4)
plt.show()

사기 거래는 평균 거래 금액이 더 높아 보입니다.

#### `TransactionDT` feature로 시간변화에 따른 분포 확인해보기

아래에서 볼 수 있듯 trian_tran과 test_tran간에는 약간의 시간 간격이 벌어져 있습니다. 이는 추후 cross validation을 해야 할지 말지에 영향을 미칩니다. 

In [ ]:
train_tran['TransactionDT'].plot(kind='hist',
                                        figsize=(15, 5),
                                        label='train',
                                        bins=50,
                                        title='Train vs Test TransactionDT distribution')
test_tran['TransactionDT'].plot(kind='hist',
                                       label='test',
                                       bins=50)
plt.legend()
plt.show()

#### train_transaction.csv의 `ProductCD` 찍어보기

In [ ]:
train_tran.head()

In [ ]:
# ProductCD의 관측 횟수
train_tran.groupby('ProductCD') \
    ['TransactionID'].count() \
    .sort_index() \
    .plot(kind='barh',
          figsize=(15, 3),
         title='Count of Observations by ProductCD')
plt.show()

# ProductCD별 사기의 비율
train_tran.groupby('ProductCD')['isFraud'] \
    .mean() \
    .sort_index() \
    .plot(kind='barh',
          figsize=(15, 3),
         title='Percentage of Fraud by ProductCD')
plt.show()

유독 'C' 코드가 사기 비중이 높습니다. 

### train_transaction.csv의 카테고리형 피쳐들 살펴보기
#### `card1`~`card6` 살펴보기

In [ ]:
card_cols = [c for c in train_tran.columns if 'card' in c]
train_tran[card_cols].head()

In [ ]:
color_idx = 0
for c in card_cols:
    if train_tran[c].dtype in ['float64','int64']:
        train_tran[c].plot(kind='hist',
                                      title=c,
                                      bins=50,
                                      figsize=(15, 2),
                                      color=color_pal[color_idx])
    color_idx += 1
    plt.show()

In [ ]:
train_transaction_fr = train_tran.loc[train_tran['isFraud'] == 1]
train_transaction_nofr = train_tran.loc[train_tran['isFraud'] == 0]

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 8))

train_transaction_fr.groupby('card4')['card4'].count().plot(kind='barh', ax=ax1, title='Count of card4 fraud')
train_transaction_nofr.groupby('card4')['card4'].count().plot(kind='barh', ax=ax2, title='Count of card4 non-fraud')
train_transaction_fr.groupby('card6')['card6'].count().plot(kind='barh', ax=ax3, title='Count of card6 fraud')
train_transaction_nofr.groupby('card6')['card6'].count().plot(kind='barh', ax=ax4, title='Count of card6 non-fraud')
plt.show()

#### `TransactionDT`와 `card`~`card6`을 groupby해보기

In [ ]:
# 올바른 card_id만 남기고 나머지는 쓰레기값으로 대체
def corret_card_id(x): 
    x=x.replace('.0','')
    x=x.replace('-999','nan')
    return x

In [ ]:
# TransactionDT를 date로 변환하고 새로운 column 추가
def definie_indexes(df):
    
    # date column 추가
    START_DATE = '2017-12-01'
    startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
    df['date'] = df['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
    
    # create card ID 
    cards_cols= ['card1', 'card2', 'card3', 'card5']
    for card in cards_cols: 
        if '1' in card: 
            df['Card_ID']= df[card].map(str)
        else : 
            df['Card_ID']+= ' '+df[card].map(str)
    
    # sort train data by Card_ID and then by transaction date 
    df= df.sort_values(['Card_ID', 'date'], ascending=[True, True])
    
    # small correction of the Card_ID
    df['Card_ID']=df['Card_ID'].apply(corret_card_id)
    
    # set indexes 
    df= df.set_index(['Card_ID', 'date'])
    return df

In [ ]:
# date column을 추가하여 기존의 train_tran 덮어쓰기
with timer('define real IDs...'):
    train_tran = definie_indexes(train_tran)

#### `add1`~`addr2` 살펴보기
`dist` feature는 단순 거리이므로 따로 분석하지 않았습니다. 

In [ ]:
# NA값 체크하기
print(' addr1 - has {} NA values'.format(train_tran['addr1'].isna().sum()))
print(' addr2 - has {} NA values'.format(train_tran['addr2'].isna().sum()))

In [ ]:
train_tran['addr1'].plot(kind='hist', bins=500, figsize=(15, 2), title='addr1 distribution')
plt.show()
train_tran['addr2'].plot(kind='hist', bins=500, figsize=(15, 2), title='addr2 distribution')
plt.show()

#### `P_emaildomain`과 `R_emaildomain` feature 살펴보기

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(18, 12))

sns.set(color_codes=True)
p_email = sns.countplot(y='P_emaildomain', data=train_tran, ax=axes[0])
r_email = sns.countplot(y='R_emaildomain', data=train_tran, ax=axes[1])
plt.tight_layout()

# 유니크한 이메일 주소의 개수
print('Count of Unique P_emaildomain : ', train_tran.P_emaildomain.nunique()) # 59 unique domains
print('Count of Unique R_emaildomain : ', train_tran.R_emaildomain.nunique()) # 59 unique domains

Gmail, Outlook, Yahoo 등의 이메일 도메인이 상위권에 올라 있지만 특이하게 anonymous.com이 눈에 띕니다.

#### `c1`~`c14` 살펴보기
c1~c14는 숫자로 된 열들이므로 pairplot을 그려 보도록 합니다. pairplot을 그려 보는 것은 특정 feature가 다른 feature과 차이가 있을 경우 유용한 방법입니다. 

In [ ]:
c_cols = [c for c in train_tran if c[0] == 'C']
print(train_tran[c_cols].head())

# 500개의 사기 및 정상 거래를 샘플링해 그래프를 그립니다.
sampled_train = pd.concat([train_tran.loc[train_tran['isFraud'] == 0].sample(500),
          train_tran.loc[train_tran['isFraud'] == 1].sample(500)])

sns.pairplot(sampled_train, 
             hue='isFraud',
            vars=c_cols)
plt.show()

#### `D1`~`D9` features에 대해서도 그래프 그려보기

In [ ]:
d_cols = [c for c in train_tran if c[0] == 'D']
print(train_tran[d_cols].head())

sns.pairplot(sampled_train, 
             hue='isFraud',
            vars=d_cols)
plt.show()

#### `M1`~`M9` feature 살펴보기
`M1`~`M9`는 T, F, NaN값을 갖고 있습니다. 

In [ ]:
m_cols = [c for c in train_tran if c[0] == 'M']
print(train_tran[m_cols].head())

(train_tran[m_cols] == 'T').sum().plot(kind='bar',
                                              title='Count of T by M column',
                                              figsize=(15, 2))
plt.show()
(train_tran[m_cols] == 'F').sum().plot(kind='bar',
                                              title='Count of F by M column',
                                              figsize=(15, 2))
plt.show()
(train_tran[m_cols].isna()).sum().plot(kind='bar',
                                              title='Count of NaN by M column',
                                              figsize=(15, 2))
plt.show()

`M0`과 `M4` column이 문제가 있음을 확인할 수 있습니다

#### `V1`~`V339` 살펴보기

In [ ]:
v_cols = [c for c in train_tran if c[0] == 'V']
print(train_tran[v_cols].head())

train_tran.describe()

In [ ]:
train_tran.info()

In [ ]:
# train_iden.info()
train_iden.head()

In [ ]:
# 그래프를 그리기 위해 v_mean column 추가하기
train_tran['v_mean'] = train_tran[v_cols].mean(axis=1)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(15, 6))
train_tran.loc[train_tran['isFraud'] == 1]['v_mean'] \
    .apply(np.log) \
    .plot(kind='hist',
          bins=100,
          title='log transformed mean of V columns - Fraud',
          ax=ax1)
train_tran.loc[train_tran['isFraud'] == 0]['v_mean'] \
    .apply(np.log) \
    .plot(kind='hist',
          bins=100,
          title='log transformed mean of V columns - Not Fraud',
          ax=ax2)
plt.show()

### train_identity.csv 분석하기

#### train_identity.csv의 `DeviceInfo` plot 그려보기

In [ ]:
# DeviceCount, DeviceInfo 두 개의 열을 갖는 DF를 만듭니다.
group = pd.DataFrame()
group['DeviceCount'] = train_iden.groupby(['DeviceInfo'])['DeviceInfo'].count()
group['DeviceInfo'] = group.index

# top 30개 Device들에 대해 sort합니다. 
group_top = group.sort_values(by='DeviceCount',ascending=False).head(30)

plt.figure(figsize=(25, 10))
sns.set(color_codes=True)
sns.set(font_scale = 1.3)
ax = sns.barplot(x="DeviceInfo", y="DeviceCount", data=group_top)
xt = plt.xticks(rotation=60)

1등이 Windows, 2등이 IOS Device, 3등이 MacOS인것을 확인할 수 있습니다. 또한 주로 desktop에서 결제를 하는 것으로 보입니다. 이제 운영체제별 사기율을 확인해 보겠습니다.

In [ ]:
# Subset dataframe
fraud = pd.DataFrame()
is_fraud = train_iden[train_tran['isFraud']==1]
fraud['DeviceCount'] = is_fraud.groupby(['DeviceInfo'])['DeviceInfo'].count()
fraud['DeviceInfo'] = fraud.index

# There are too many Devices, so we will subset the top 20
group_top = fraud.sort_values(by='DeviceCount',ascending=False).head(20)

plt.figure(figsize=(25, 10))
sns.set(color_codes=True)
sns.set(font_scale = 1.3)
ax = sns.barplot(x="DeviceInfo", y="DeviceCount", data=group_top)

font_size= {'size': 'x-large'}
ax.set_title("Fraud transactions by OS", **font_size)
xt = plt.xticks(rotation=60)

위 그래프와 크게 다르지는 않습니다. 순서대로 Windows, IOS 순으로 사기 거래가 잦습니다. 특히 Windows OS에서 그 비중이 높은 것을 알 수 있습니다.

#### `DeviceType` 살펴보기

In [ ]:
# 사기여부와 함께 DeviceType feature 분석을 위해 `isFraud` column 추가
train_identity_ = train_iden.merge(train_tran[['TransactionID', 'TransactionDT', 'isFraud']], 
                                   on=['TransactionID'])
test_identity_ = test_iden.merge(test_tran[['TransactionID', 'TransactionDT']], 
                                 on=['TransactionID'])

In [ ]:
train_identity_.head()

In [ ]:
train_identity_.groupby('DeviceType') \
    .mean()['isFraud'] \
    .sort_values() \
    .plot(kind='barh',
          figsize=(15, 5),
          title='Percentage of Fraud by Device Type')
plt.show()

사기 거래의 비율은 mobile에서 더 높은 것을 알 수 있습니다.

#### `id01`~`id_38` 분석해보기

In [ ]:
#Exploration id_12 - id_38
id01_loc = train_iden.columns.get_loc("id_01")
id38_loc = train_iden.columns.get_loc("id_38")
df_id = train_iden.iloc[id01_loc:id38_loc+1] #subset dataframe id12-id19

In [ ]:
df_id.dtypes # id features의 데이터 타입 모두 출력

In [ ]:
df_id.head(10)

`id` features에는 다양한 데이터 타입이 섞여 있으며, 대부분 NaN값을 갖고 있습니다. `id_30`은 운영체제이고, `id_31`은 사용하는 웹 브라우저, `id_33`은 기기 해상도를 나타내는 것으로 보입니다. `id_30`과 `id_31`을 자세히 살펴 보도록 합시다.

#### `id_30` 그려보기

In [ ]:
# First create a dataframe with 2 cols: device info and the count by device
group_id30 = pd.DataFrame()
group_id30['id_30Count'] = df_id.groupby(['id_30'])['id_30'].count()
group_id30['id_30'] = group_id30.index

# There are too many addr, so we will subset the top 20
group_top_id30 = group_id30.sort_values(by='id_30Count',ascending=False).head(20)

plt.figure(figsize=(25, 10))
sns.set(color_codes=True)
sns.set(font_scale = 1.3)
ax = sns.barplot(x="id_30", y="id_30Count", data=group_top_id30)
xt = plt.xticks(rotation=60)

거래는 Windows 10, IOS 11.2.2, Android 7.0 순으로 많이 이루어졌습니다.

In [ ]:
# 운영체제, 브라우저별 사기율을 보기 위해 isFraud column 추가
group_id = train_iden.merge(train_tran[['TransactionID', 'isFraud']], 
                                 on=['TransactionID'])

In [ ]:
group_id.head()

In [ ]:
group_id.groupby('id_30') \
    .mean()['isFraud'] \
    .sort_values() \
    .plot(kind='barh',
          figsize=(15, 25),
          title='Percentage of Fraud by OS Type')
plt.show()

(featured) 기타 및 구형 안드로이드 OS에서 유독 사기 거래의 비율이 높습니다. `id_31`에 대해서도 같은 작업을 반복해 보겠습니다

#### `id_31` 그려보기

거래는 또한 Chrome 62.0, Chrome 62.0 for Android, mobile Safari 11.0 브라우저 순으로 이루어졌습니다. 이제 브라우저별 사기율도 확인해 보겠습니다.

In [ ]:
group_id.groupby('id_31') \
    .mean()['isFraud'] \
    .sort_values() \
    .plot(kind='barh',
          figsize=(15, 45),
          title='Percentage of Fraud by Browser Type')
plt.show()

icedragon(Firefox 기반 오픈 소스 웹 브라우저)가 제일 사기를 당할 확률이 높았고, 그 뒤를 이어 모질라/파이어폭스가 잇고 있습니다. 크롬 브라우저의 경우 버전이 낮을수록 사기 거래를 당하기 쉬웠습니다.

#### `id_33` 그려보기
`id_33`은 사용자의 기기 해상도로 사기 거래와 관련이 없어 보일 수 있습니다. 그러나 해상도가 낮을수록 사용자 기기의 성능이 낮고, 그에 따라 낮은 버전의 프로그램을 사용할 것이라는 생각이 들었습니다. 별 의미가 없어 보일 수 있지만 `id_33`에 대해서도 isFraud에 대한 비율을 비교해 보도록 하겠습니다

In [ ]:
group_id.groupby('id_33').mean()['isFraud'].sort_values()
group_id.sort_values('id_33', ascending=False).head(20).plot(kind='barh', \
                     figsize=(15, 75), title='Percentage of Fraud by Resolution')
plt.show()

In [ ]:
# 메모리 사용량 확인하기
!pip install memory_profiler

In [ ]:
%load_ext memory_profiler

In [ ]:
%memit

In [ ]:
# 메모리 해제
import gc

del [[train_tran, train_iden, test_tran, test_iden]]
gc.collect()

### 원하는 feature의 column만 뽑아 새로운 DF 구성하기

위에서 열심히 진행한 EDA 과정을 통해 아래의 features를 뽑았습니다. 

* from `train_transaction.csv`
    - TransactionAMT
    - ProductCD
    - card1~card6 
    - P_emaildomain과 R_emaildomain feature
    - M1 ~ M9 / isFraud
* from `train_identity.csv`
    - DeviceType, DeviceInfo
    - id_30 , id_31, id_33

In [ ]:
# 원하는 feature의 열들만 뽑아 새로운 DF로 구성합니다. 
cols= ['isFraud','TransactionDT','TransactionAmt','ProductCD', 'P_emaildomain', 'R_emaildomain']
cards_cols= ['card1', 'card2', 'card3','card4','card5', 'card6']
train[cols].head(30)

## AutoEncoder 학습하기
- WIP(non-fraud data로 학습해야 함.)

In [ ]:
import os

import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import seaborn as sns
import collections

from tqdm import tqdm, tqdm_notebook


from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,
                             roc_curve, recall_score, classification_report, f1_score)
from sklearn.metrics import accuracy_score, precision_score
                            
import torch.nn as nn
from torch.autograd import Variable as V
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
%matplotlib inline
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.models import HoverTool, NumeralTickFormatter
from bokeh.palettes import Set3_12
from bokeh.transform import jitter

import gc
gc.enable()

In [ ]:
# Training epochs
epochs=10

In [ ]:
train_transaction = pd.read_csv(os.path.join(dataset_dir, 'train_transaction.csv'), index_col='TransactionID')
train_identity = pd.read_csv(os.path.join(dataset_dir, 'train_identity.csv'), index_col='TransactionID')
test_transaction = pd.read_csv(os.path.join(dataset_dir, 'test_transaction.csv'), , index_col='TransactionID')
test_identity = pd.read_csv(os.path.join(dataset_dir, 'test_identity.csv'), index_col='TransactionID')
sample_submission = pd.read_csv(os.path.join(dataset_dir, 'sample_submission.csv'), index_col='TransactionID')

train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

print(train.shape)
print(test.shape)

### Drop columns(to drop NaN values) and Standard scaling

In [ ]:
# Drop columns
def dropper(column_name, train, test):
    train = train.drop(column_name, axis=1)
    test = test.drop(column_name, axis=1)
    return train, test

del_columns = ['TransactionDT']
for col in del_columns:
    train, test = dropper(col, train, test)

def scaler(scl, column_name, data):
    data[column_name] = scl.fit_transform(data[column_name].values.reshape(-1,1))
    return data

scl_columns = ['TransactionAmt', 'card1', 'card3', 'card5', 'addr1', 'addr2']
for col in scl_columns:
    train = scaler(StandardScaler(), col, train)
    test = scaler(StandardScaler(), col, test)

In [ ]:
train.head()

In [ ]:
"""
#TODO: Learning should be done by using non fraud data
train = train[train['isFraud'] == 0]
train_fraud = train[train['isFraud'] == 1].copy()
"""

### Build AutoEncoder

#### Preprocessing

In [ ]:
y_train = train['isFraud'].copy()
del train_transaction, train_identity, test_transaction, test_identity

# Drop target
X_train = train.drop('isFraud', axis=1)
#X_train_fraud = train_fraud.drop('isFraud', axis=1)
X_test = test.copy()

del train, test
    
# TODO: change methods
# Fill in NaNs
X_train = X_train.fillna(-999)
#X_train_fraud = X_train_fraud.fillna(-999)
X_test = X_test.fillna(-999)

# TODO: change to Label Count Endocing
# Label Encoding
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values)) #+ list(X_train_fraud[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        #X_train_fraud[f] = lbl.transform(list(X_train_fraud[f].values)) 
        X_test[f] = lbl.transform(list(X_test[f].values)) 
        
gc.collect()

In [ ]:
print(X_train.head())
#print(X_train_fraud.head())
print(X_test.head())

In [ ]:
"""
    params:
        data : data desired to be split
        ratio : validation ratio for split
        
    output:
        train_data, validation_data
"""

def splitter(data, ratio=0.2):
    num = int(ratio*len(data))
    return data[num:], data[:num]

X_train, X_val = splitter(X_train)
y_train, y_val = splitter(y_train)

# Check number of data
print(len(X_train), len(X_val), len(y_train), len(y_val))

In [ ]:
xtr = torch.FloatTensor(X_train.values)
xts = torch.FloatTensor(X_test.values)
# X_val: validation data for isFraud == 0
xvl = torch.FloatTensor(X_val.values) 
# X_train_fraud: validation data for isFraud == 1
#xvt = torch.FloatTensor(X_train_fraud.values)

xdl = DataLoader(xtr,batch_size=1000)
tdl = DataLoader(xts,batch_size=1000)
vdl = DataLoader(xvl,batch_size=1000)
#fdl = DataLoader(xvt,batch_size=1000)

print(len(X_train.values), len(X_test.values), len(X_val.values)) #, len(X_train_fraud))
gc.collect()

In [ ]:
# Build AE class

class AutoEncoder(nn.Module):
    def __init__(self, length):
        super().__init__()
        self.lin1 = nn.Linear(length,20)
        self.lin2 = nn.Linear(20,10)
        self.lin7 = nn.Linear(10,20)
        self.lin8 = nn.Linear(20,length)
        
        self.drop2 = nn.Dropout(0.05)
        
        self.lin1.weight.data.uniform_(-2,2)
        self.lin2.weight.data.uniform_(-2,2)
        self.lin7.weight.data.uniform_(-2,2)
        self.lin8.weight.data.uniform_(-2,2)

    def forward(self, data):
        x = F.tanh(self.lin1(data))
        x = self.drop2(F.tanh(self.lin2(x)))
        x = F.tanh(self.lin7(x))
        x = self.lin8(x)
        return x
    
def score(x):
    y_pred = model(V(x))
    x1 = V(x)
    return loss(y_pred,x1).item()

In [ ]:
model = AutoEncoder(len(X_train.columns))
loss=nn.MSELoss()
learning_rate = 1e-2
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)
print(model)

# Utilize a named tuple to keep track of scores at each epoch
model_hist = collections.namedtuple('Model','epoch loss val_loss')
model_loss = model_hist(epoch = [], loss = [], val_loss = [])

In [ ]:
# Define training routine

def train(epochs, model, model_loss):
    try: c = model_loss.epoch[-1]
    except: c = 0
    for epoch in tqdm_notebook(range(epochs),position=0, total = epochs):
        losses=[]
        dl = iter(xdl)
        for t in range(len(dl)):
            # Forward pass: compute predicted y and loss by passing x to the model.
            xt = next(dl)
            y_pred = model(V(xt))
            
            l = loss(y_pred,V(xt))
            losses.append(l)
            optimizer.zero_grad()

            # Backward pass: compute gradient of the loss with respect to model parameters
            l.backward()

            # Calling the step function on an Optimizer makes an update to its parameters
            optimizer.step()
            
        val_dl = iter(tdl)
        val_scores = [score(next(val_dl)) for i in range(len(val_dl))]
        
        model_loss.epoch.append(c+epoch)
        model_loss.loss.append(l.item())
        model_loss.val_loss.append(np.mean(val_scores))
        print(f'Epoch: {epoch}   Loss: {l.item():.4f}    Val_Loss: {np.mean(val_scores):.4f}')

train(model=model, epochs=epochs, model_loss=model_loss)

#### Check Loss/Validation Loss

In [ ]:
x = np.linspace(0, epochs-1, epochs)
print(model_loss.loss)
print(model_loss.val_loss)
print(x)
plt.plot(x, model_loss.loss, label="loss")
plt.legend()
plt.show()

plt.plot(x, model_loss.val_loss, label="val_loss")
plt.legend()
plt.show()

gc.collect()

#### Validation

In [ ]:
# Iterate through the dataloader and get predictions for each batch of the test set.
p = iter(vdl)
preds = np.vstack([model(V(next(p))).cpu().data.numpy() for i in range(len(p))])

# Create a pandas DF that shows the Autoencoder MSE vs True Labels
error_nonfraud = np.mean(np.power((X_val-preds),2), axis=1)
"""
p = iter(fdl)
preds = np.vstack([model(V(next(p))).cpu().data.numpy() for i in range(len(p))])
error_fraud = np.mean(np.power((X_train_fraud-preds),2), axis=1)

pd.DataFrame(error_fraud)
"""
error_df = pd.DataFrame(data = {'error':error_nonfraud,'true':y_val})

error_df.groupby('true')['error'].describe().reset_index()

#### Calculate ROC Curve

In [ ]:
fpr, tpr, thresholds = roc_curve(error_df.true, error_df.error)
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, label='ROC curve (area = {})'.format(roc_auc))
plt.legend()
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)

In [ ]:
temp_df = error_df[error_df['true'] == 0]
threshold = temp_df['error'].mean() + temp_df['error'].std()
print(f'Threshold: {threshold:.3f}')

#### Calculate Recall F1-Score

In [ ]:
y_pred = [1 if e > threshold else 0 for e in error_df.error.values]
print(classification_report(error_df.true.values,y_pred))

In [ ]:
# Ploting Precision Recall
conf_matrix = confusion_matrix(error_df.true, y_pred)

sns.set(font_scale = 1.2)
plt.figure(figsize=(10, 10))
sns.heatmap(conf_matrix, xticklabels=['Not Fraud','Fraud'], yticklabels=['Not Fraud','Fraud'], annot=True, fmt="d");
plt.title("Confusion matrix")
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()

In [ ]:
# Ploting Precision Recall for each thresholds
# Changing the threshold from threshold_min to threshold_max.
plt.figure(figsize=(12, 12))
m = []
threshold_min = threshold * 0.99
threshold_max = threshold * 1.01

for thresh in np.linspace(threshold_min, threshold_max):
    y_pred = [1 if e > thresh else 0 for e in error_df.error.values]
    conf_matrix = confusion_matrix(error_df.true, y_pred)
    m.append((conf_matrix,thresh))
    
count = 0
for i in range(3):
    for j in range(3):
        plt.subplot2grid((3, 3), (i, j))
        sns.heatmap(m[count][0], xticklabels=['Not Fraud','Fraud'], yticklabels=['Not Fraud','Fraud'], annot=True, fmt="d");
        plt.title(f"Threshold - {m[count][1]:.3f}")
        plt.ylabel('True class')
        plt.xlabel('Predicted class')
        plt.tight_layout()
        count += 1
plt.show()

In [ ]:
# Iterate through the dataloader and get predictions for each batch of the test set.
p = iter(tdl)
preds = np.vstack([model(V(next(p))).cpu().data.numpy() for i in range(len(p))])

# Create a pandas DF that shows the Autoencoder MSE vs True Labels
error = np.mean(np.power((X_test-preds),2), axis=1)

In [ ]:
def min_max_normalization(x):
    x_min = x.min()
    x_max = x.max()
    x_norm = (x-x_min) / (x_max-x_min)
    return x_norm

# min max normalization
#error_df = pd.DataFrame(data={'isFraud':min_max_normalization(error)})
error_df = pd.DataFrame(data={'isFraud':error})

print("Num data: " + str(len(error_df)))
print("Beyond threshold num data: " + str(len(error_df[error_df['isFraud'] > threshold])))
#error_df[error_df['isFraud'] > threshold]

x_min = 3600000
x_max = 4200000
plt.hlines(threshold, x_min, x_max, "black")
plt.plot(error_df, alpha=0.3)
plt.show()

In [ ]:
error_df = pd.DataFrame(data={'isFraud':min_max_normalization(error)})
error_df.head()

In [ ]:
sample_submission['isFraud'] = error_df
sample_submission.to_csv('simple_autoencoder.csv')